In [1]:
import collections
import math
import random

### Get a sieve of all numbers $n$ to indicate whether they are prime, with $0 \leq n \leq 10^7$

In [2]:
def get_primes_eratosthenes_sieve(n):
    is_prime = [1] * (n + 1)
    p = 2
    is_prime[0] = 0
    is_prime[1] = 0
    while p * p <= n:
        if is_prime[p]:
            for i in range(p * 2, n + 1, p):
                is_prime[i] = 0
        p += 1
    prime_list = []
    for i in range(n + 1):
        if is_prime[i]:
            prime_list.append(i)
    return is_prime

is_prime = get_primes_eratosthenes_sieve(10 ** 7)

### Use Pollard's Rho algorithm for finding prime factorizations for any $n$

In [3]:
# https://github.com/zhangbo2008/python_algorithm2/blob/c53669703b957a079f100c12711f86f5fc2f9389/algorithms/factorization/pollard_rho.py
def pollard_rho_prime_factorization(x):
    def f(x):
        return x * x + 1

    def rho(n, x1=2, x2=2):
        if n % 2 == 0:
            return 2
        i = 0
        while True:
            x1 = f(x1) % n
            x2 = f(f(x2)) % n
            divisor = math.gcd(abs(x1 - x2), n)
            i += 1
            if divisor != 1:
                break
            if i > 500:
                x1 = random.randint(1, 10)
                x2 = random.randint(1, 10)
                i = 0
        return divisor

    def pollard_rho_rec(x, factors):
        if x == 1:
            return

        if is_prime[x]:
            factors.append(x)
            return

        divisor = rho(x, random.randint(1, 10), random.randint(1, 10))
        pollard_rho_rec(divisor, factors)
        pollard_rho_rec(x // divisor, factors)

    if x == 1 or x == 0:
        return [(x, 1)]
    factors = []
    pollard_rho_rec(x, factors)
    return collections.Counter(factors)

### Find multiplicative inverse $x$ satisfying $xa \equiv 1\,(\text{mod b})$ and $\text{gcd}(a, b) = 1$

In [4]:
def mul_inv(a, b):
    if b == 1:
        return 1
    b_0 = b
    x_0, x_1 = 0, 1
    while a > 1:
        x_0, x_1 = x_1 - (a // b) * x_0, x_0
        a, b = b, a % b
    if x_1 < 0:
        x_1 += b_0
    return x_1

### Compute $F(n)=\sum_{i=2}^n S(i!)$ by iterating through each $i$ and computing its prime factorization, and then updating the value of $S$ computed for the previous iteration using the observation $S(i!) = \prod{(2\cdot \alpha_j + 1)}$ where $i! = \prod{p_j^{\alpha_j}}$

In [5]:
def F(n, M):
    factors_count = {p: 0 for p in [n for n in range(len(is_prime)) if is_prime[n] == 1]}
    S_sum = S_i = 3  # Starting with S(2!) = S(2) = 2^w(1) + 2^w(2) = 2^0 + 2^1 = 3
    factors_count[2] = 1
    for i in range(3, n + 1):
        prime_factorization_i = pollard_rho_prime_factorization(i)
        for p in prime_factorization_i:
            S_i = (S_i * (2 * (factors_count[p] + prime_factorization_i[p]) + 1) * mul_inv(2 * factors_count[p] + 1, M)) % M
            factors_count[p] += prime_factorization_i[p]
        S_sum = (S_sum + S_i) % M

        # Display progress of function execution for large values of N
        if i % (10 ** 5) == 0:
            print('Progress:', i // (10 ** 5), '%', end='\r', flush=True)
        
    return S_sum % M

F(10 ** 7, 10 ** 9 + 87)

416146418